In [ ]:
! pip install sentence-transformers

In [ ]:
import pandas as pd
import math
import scipy
from copy import deepcopy
from sentence_transformers import SentenceTransformer
import re
import numpy as np

In [7]:
embedder = SentenceTransformer('roberta-base-nli-stsb-mean-tokens')

100%|██████████| 459M/459M [00:19<00:00, 23.8MB/s]


In [ ]:
''' Retrofit word vectors to a lexicon '''
def retrofit(wordVecs, lexicon, numIters):
    newWordVecs = deepcopy(wordVecs)
    wvVocab = set(newWordVecs.keys())
    loopVocab = wvVocab.intersection(set(lexicon.keys()))
    for it in range(numIters):
    # loop through every node also in ontology (else just use data estimate)
        for word in loopVocab:
            wordNeighbours = set(lexicon[word]).intersection(wvVocab)
            numNeighbours = len(wordNeighbours)
            #no neighbours, pass - use data estimate
            if numNeighbours == 0:
                continue
            # the weight of the data estimate is the number of neighbours
            newVec = numNeighbours * wordVecs[word]
            # loop over neighbours and add to new vector (currently with weight 1)
            for ppWord in wordNeighbours:
                newVec += newWordVecs[ppWord]
            newWordVecs[word] = newVec/(2*numNeighbours)
    return newWordVecs

In [ ]:
example_papers = pd.DataFrame([
    {"paper_id": "1",
     "text_to_embed": "Standard word embeddings lack the possibility to distinguish senses of a word by projecting them to exactly one vector. This has a \
negative effect particularly when computing similarity scores between words using standard vector-based similarity measures such as \
cosine similarity. We argue that minor senses play an important role in word similarity computations, hence we use an unsupervised \
sense inventory resource to retrofit monolingual word embeddings, producing sense-aware embeddings. Using retrofitted sense-aware \
embeddings, we show improved word similarity and relatedness results on multiple word embeddings and multiple established word \
similarity tasks, sometimes up to an impressive margin of +0.15 Spearman correlation score.",
    "references": ["4", "5", "7"]},
    {"paper_id": "2",
     "text_to_embed": "Training large deep neural networks on massive datasets is  computationally very challenging. There has been recent surge in interest in using large batch stochastic optimization methods to tackle this issue. The most prominent algorithm in this line of research is LARS, which by  employing layerwise adaptive learning rates trains ResNet on ImageNet in a few minutes. However, LARS performs poorly for attention models like BERT, indicating that its performance gains are not consistent across tasks. In this paper, we first study a principled layerwise adaptation strategy to accelerate training of deep neural networks using large mini-batches. Using this strategy, we develop a new layerwise adaptive large batch optimization technique called LAMB; we then provide convergence analysis of LAMB as well as LARS, showing convergence to a stationary point in general nonconvex settings. Our empirical results demonstrate the superior performance of LAMB across various tasks such as BERT and ResNet-50 training with very little hyperparameter tuning. In particular, for BERT training, our optimizer enables use of very large batch sizes of 32868 without any degradation of performance.  By increasing the batch size to the memory limit of a TPUv3 Pod, BERT training time can be reduced from 3 days to just 76 minutes.",
    "references": ["1", "5", "4", "6"]},
    {"paper_id": "3",
     "text_to_embed": 'Noisy labels are very common in real-world training data, which lead to poor generalization on test data because of overfitting to the noisy labels. In this paper, we claim that such overfitting can be avoided by "early stopping" training a deep neural network before the noisy labels are severely memorized. Then, we resume training the early stopped network using a "maximal safe set," which maintains a collection of almost certainly true-labeled samples at each epoch since the early stop point. Putting them all together, our novel two-phase training method, called Prestopping, realizes noise-free training under any type of label noise for practical use. Extensive experiments using four image benchmark data sets verify that our method significantly outperforms four state-of-the-art methods in test error by 0.4–8.2 percent points under existence of real-world noise.',
    "references": ["1", "2", "8"]},
    {"paper_id": "4",
     "text_to_embed": 'The recently developed link between strongly overparametrized neural networks (NNs) and kernel methods has opened a new way to understand puzzling features of NNs, such as their convergence and generalization behaviors. In this paper, we make the bias of initialization on strongly overparametrized NNs under gradient descent explicit. We prove that fully-connected wide ReLU-NNs trained with squared loss are essentially a sum of two parts: The first is the minimum complexity solution of an interpolating kernel method, while the second contributes to the test error only and depends heavily on the initialization. This decomposition has two consequences: (a) the second part becomes negligible in the regime of small initialization variance, which allows us to transfer generalization bounds from minimum complexity interpolating kernel methods to NNs; (b) in the opposite regime, the test error of wide NNs increases significantly with the initialization variance, while still interpolating the training data perfectly. Our work shows that -- contrary to common belief -- the initialization scheme has a strong effect on generalization performance, providing a novel criterion to identify good initialization strategies.',
    "references": ["7", "3"]},
    {"paper_id": "5",
     "text_to_embed": 'User generated content contains opinionated texts not only in dominant languages (like English) but also less dominant languages( like Amharic). However, negation handling techniques that supports for sentiment detection is not developed in such less dominant language(i.e. Amharic). Negation handling is one of the challenging tasks for sentiment classification. Thus, this work builds negation handling schemes which enhances Amharic Sentiment classification.  The proposed Negation Handling framework combines the lexicon based approach and character ngram based machine learning model.  The performance of framework is evaluated using the annotated Amharic News Comments. The system is outperforming the best of all models and the baselines by an accuracy of 98.0. The result is compared with the baselines (without negation handling and word level ngram model).',
    "references": ["7", "3", "4"]},
    {"paper_id": "6",
     "text_to_embed": 'Non-negative matrix factorization (NMF) is a highly celebrated algorithm for matrix decomposition that guarantees strictly non-negative factors. The underlying optimization problem is computationally intractable, yet in practice gradient descent based solvers often find good solutions. This gap between computational hardness and practical success mirrors recent observations in deep learning, where it has been the focus of extensive discussion and analysis. In this paper we revisit the NMF optimization problem and analyze its loss landscape in non-worst-case settings. It has recently been observed that gradients in deep networks tend to point towards the final minimizer throughout the optimization. We show that a similar property holds (with high probability) for NMF, provably in a non-worst case model with a planted solution, and empirically across an extensive suite of real-world NMF problems. Our analysis predicts that this property becomes more likely with growing number of parameters, and experiments suggest that a similar trend might also hold for deep neural networks --- turning increasing data sets and models into a blessing from an optimization perspective.',
    "references": ["3", "5", "7", "8"]},
    {"paper_id": "7",
     "text_to_embed": 'It would be desirable for a reinforcement learning (RL) based agent to learn behaviour by merely watching a demonstration.  However, defining rewards that facilitate this goal within the RL paradigm remains a challenge. Here we address this problem with Siamese networks, trained to compute distances between observed behaviours and the agent’s behaviours. Given a desired motion such Siamese networks can be used to provide a reward signal to an RL agent via the distance between the desired motion and the agent’s motion. We experiment with an RNN-based comparator model that can compute distances in space and time between motion clips while training an RL policy to minimize this distance.  Through experimentation, we have had also found that the inclusion of multi-task data and an additional image encoding loss helps enforce the temporal consistency.  These two components appear to balance reward for matching a specific instance of a behaviour versus that behaviour in general. Furthermore, we focus here on a particularly challenging form of this problem where only a single demonstration is provided for a given task – the one-shot learning setting. We demonstrate our approach on humanoid agents in both 2D with 10 degrees of freedom (DoF) and 3D with 38 DoF.',
   "references": ["3", "2"]},
   {"paper_id": "8",
    "text_to_embed": 'Silo discharging and monitoring the process for \
industrial or research application depend on computerized \
segmentation of different parts of images such as stagnant and \
flowing zones which is the toughest task. X-ray Computed \
Tomography (CT) is one of a powerful non-destructive technique \
for cross-sectional images of a 3D object based on X-ray \
absorption. CT is the most proficient for investigating different \
granular flow phenomena and segmentation of the stagnant zone \
as compared to other imaging techniques. In any case, manual \
segmentation is tiresome and erroneous for further investigations. \
Hence, automatic and precise strategies are required. In the \
present work, a U-net architecture is used for segmenting the \
stagnant zone during silo discharging process. This proposed \
image segmentation method provides fast and effective outcomes \
by exploiting a convolutional neural networks technique with an \
accuracy of 97 percent',
   "references": []},
])

In [10]:
example_papers

,paper_id,text_to_embed,references
0,1,Standard word embeddings lack the possibility ...,"[4, 5, 7]"
1,2,Training large deep neural networks on massive...,"[1, 5, 4, 6]"
2,3,Noisy labels are very common in real-world tra...,"[1, 2, 8]"
3,4,The recently developed link between strongly o...,"[7, 3]"
4,5,User generated content contains opinionated te...,"[7, 3, 4]"
5,6,Non-negative matrix factorization (NMF) is a h...,"[3, 5, 7, 8]"
6,7,It would be desirable for a reinforcement lear...,"[3, 2]"
7,8,Silo discharging and monitoring the process fo...,[]


In [ ]:
def preprocess_sentence(sentence):
    s = re.sub(r'\.','',sentence)
    return s

def split_text(text):
    initial_split = text.split(".")
    return [preprocess_sentence(s) for s in initial_split if s != ""]

In [ ]:
def get_average_sentences_embedding(sentences, model=embedder):
    embeddings = model.encode(sentences)
    return np.mean(embeddings, axis=0)

In [ ]:
example_papers["paper_embeddings"] = example_papers.apply(lambda x: get_average_sentences_embedding(split_text(x.text_to_embed)), axis=1) 

In [18]:
example_papers

,paper_id,text_to_embed,references,paper_embeddings
0,1,Standard word embeddings lack the possibility ...,"[4, 5, 7]","[1.1414502, -0.05774516, 0.15244266, 0.4040164..."
1,2,Training large deep neural networks on massive...,"[1, 5, 4, 6]","[-0.016116971, -0.19592582, -0.38806283, 0.238..."
2,3,Noisy labels are very common in real-world tra...,"[1, 2, 8]","[0.28780848, 0.004381097, -0.18965016, -0.4036..."
3,4,The recently developed link between strongly o...,"[7, 3]","[0.23816547, -0.36646232, -0.35332927, 0.35632..."
4,5,User generated content contains opinionated te...,"[7, 3, 4]","[0.3065349, -0.12176431, 0.16986303, -0.072551..."
5,6,Non-negative matrix factorization (NMF) is a h...,"[3, 5, 7, 8]","[0.6811958, -0.09238892, -0.033893496, 0.00385..."
6,7,It would be desirable for a reinforcement lear...,"[3, 2]","[0.10984958, -0.40576145, 0.056548636, 0.37904..."
7,8,Silo discharging and monitoring the process fo...,[],"[0.0148606235, -0.23329356, -0.15579127, -0.02..."


In [ ]:
def normalize_embedding(embedding):
    embedding = np.array(embedding)
    return embedding / math.sqrt((embedding**2).sum() + 1e-6)

In [ ]:
def build_lexicon_and_corpus(papers):
    lexicon = {}
    corpus = {}
    for pi, pref, pemb in zip(papers.paper_id, papers.references, papers.paper_embeddings):
        lexicon[pi] = pref
        corpus[pi] = normalize_embedding(pemb)
    return lexicon, corpus

In [ ]:
lexicon, corpus = build_lexicon_and_corpus(example_papers)

In [ ]:
retrofitted = retrofit(corpus, lexicon, 10)

In [ ]:
assert (retrofitted['8'] == corpus['8']).all()

In [ ]:
assert not (retrofitted['1'] == corpus['1']).all()